# Banc Statique <br>
### STAGE ANCHES <br>
Camille Urban <br>
24/04/2024

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cv2
from skimage.io import imread
from skimage.transform import rotate
# from im_rotate import im_rotate
from scipy import ndimage, datasets
import os
from scipy.stats import linregress

In [2]:
%matplotlib
%matplotlib

Using matplotlib backend: <object object at 0x000002AD5F136210>
Using matplotlib backend: TkAgg


## Paramètres du Banc

In [3]:
# SENSIBILITE CAPTEUR
S_force_bs = 29.67      # bs pour banc statique, sensibilité en N/mV
G_force_bs = 100        # gain du banc de mesure

# cannal d'acquisition
# Cannal 2 

## Traitement données 

In [5]:
plt.close('all')

# matrice vide
data = []
incertitudes = []
ecarts_types = []


# labels 
anches = ['Force 1', 'Force 3 1/2', 'Force 3 1/2', 'Force 5']
positions = [0.2, 0.4, 0.6]

# palette de couleurs
palette_couleurs = plt.cm.viridis

plt.figure(f'Répétabilité')
plt.title(f'Répétabilité')

# test de répétabilité effectue sur 4 anches différentes
for i in range (4):
    couleur = palette_couleurs(i/4)
    # recuperation donnees
    # data = pd.read_table(f'Banc statique/2024.04.17-repetabilite/repetabilite_{i+1}/Anche{i+1}.txt', sep=' ', header=1).values # PC TRAVAIL
    data = pd.read_table(f'2024.04.17-repetabilite/repetabilite_{i+1}/Anche{i+1}.txt', sep=' ', header=1).values  # PC Perso
    means =  []

    # repetabilite pour 3 positions sur la largeur d anche
    for m in range (3) :
        
        if (data[:, 0] == m).any() :
            rep = data[data[:, 0] == m, 1]
            force = data[data[:, 0] == m, 4]
        mean = np.mean(force)
        means.append(mean)
        ecart_type = np.std(force, ddof=1)
        ecarts_types.append(ecart_type)

    plt.plot(positions, means, '.-', label=f'{anches[i]}', color = couleur)
    plt.errorbar(positions, means, yerr=ecart_type, fmt='.', color=couleur, capsize=5)    
    coefficients = np.polyfit(positions, means, 1)       # courbe paramétrique 
    polynome = np.poly1d(coefficients) 
    x = np.linspace(min(positions), max(positions), 100)
    y = polynome(x)
    print('pente', round(coefficients[0], 2))
    # plt.plot(x, y, color=couleur, label=f'Courbe paramétrique ajustée de pente {coefficients[0]}')
plt.xlabel('Position y (mm)')
# plt.ylabel('Force (N)')
plt.legend(title = 'Numéro d\'anche')
plt.grid(True) 


plt.show()

pente 1.4
pente 2.24
pente 2.74
pente 3.78


In [6]:
plt.close('all')

# matrice vide
data = []
A = []

# labels 
anches = ['Force 1', 'Force 3 1/2', 'Force 3 1/2', 'Force 5']
force_appui = [0.5, 0.7, 0.9, 1.1]

# palette de couleurs
palette_couleurs = plt.cm.viridis


# profil d'anche
for i in range (4):
    plt.figure(f'Profil d\'anche num {i+1}')
    plt.title(f'Profil d\'anche n°{i+1} ({anches[i]})')
    for j in range (4):
        couleur = palette_couleurs(j/4)
        # recuperation donnees
        # data = pd.read_table(f'Banc statique/2024.04.17-profil_anche/profil_{i+1}_{j+1}/data{j+1}.txt', sep=' ', header=1).values
        data = pd.read_table(f'2024.04.17-profil_anche/profil_{i+1}_{j+1}/data{j+1}.txt', sep=' ', header=1).values
        # print(data)
        position = data[:,0]
        force = data[:,2]
        plt.plot(position, force, '.-', label=f'position d\'appui {force_appui[j]} mm', color = couleur)
        
        coefficients = np.polyfit(position, force, 4)       # courbe paramétrique 
        polynome = np.poly1d(coefficients)
        x = np.linspace(min(position), max(position), 100)
        y = polynome(x)
        # print(coefficients)
        plt.plot(x, y, color=couleur, label='Courbe paramétrique ajustée')
        # plt.errorbar(position, force, yerr=ecart_type_tot, fmt='.', color=couleur, capsize=5)  # barres d'erreurs
        plt.xlabel('y (mm)')
        plt.ylabel('Force (N)')
        plt.legend()
        plt.grid(True)
        

plt.show()

In [9]:
# graphique 2D de la force appliquée en chaque point de mesure
plt.close('all')

import matplotlib.colors as mcolors


# dossier = "Banc statique/2024.04.23/" # ordi pro
dossier = "2024.04.23/"
savename = "pentes"
pentes = []
mymap = plt.get_cmap("Spectral")
# mymap = plt.get_cmap("twilight")

Forces = ['Force 1', 'Force 3 1/2', 'Force 3 1/2', 'Force 5']


for j in range (4):
    plt.figure(f'Anche {j+1}')
    plt.title(f'Force mesurée en fonction de l\'appui vertical (position en z) pour chaque \n position y sur la largeur d\'anche - Anche n°{j+1} ({Forces[j]})', fontsize=16)
    # recuperation donnees
    data = pd.read_table(dossier + f'anche{j+1}/data_anche{j+1}.txt', sep=' ', header=0).values 
    indices_tries = np.lexsort((data[:, 1], data[:, 0]))
    data = data[indices_tries] 
    # print(data)
    y = data[:,0]
    z = data[:,1]
    force = data[:,2]
    N = 6
    
    for i in range (0, len(data), N):
        # palette de couleurs
        couleur = mymap(i / len(data))
        
        # trace les courbes de la force mesurées en fonction de l'appuie pour chaque position sur la largeur d'anche
        plt.plot(z[i:i+N], force[i:i+N], marker='*', color=couleur, label=f'position d\'appui {np.round(y[i], 2)} mm')
        
        # extractions des paramètres de la courbe paramétrique
        coefficients = np.polyfit(z[i+1:i+N], force[i+1:i+N], 1)
        pente = coefficients[0]
        pente = round(pente, 2)
        # ordonnee_origine = coefficients[1]
        # ordonnee_origine = round(ordonnee_origine, 2)
        pentes.append(['anche', j, 'position y', y[i], 'pente', pente])
        
        # trace la courbe paramétrique sur la section qui nous intéresse
        # plt.plot(x=data_a[i:i+4,1], np.polyval(coefficients, data_a[i:i+4,1]), label=f'Droite de régression (pente={pente:.2f}, ordonnée origine={ordonnee_origine:.2f})')
        plt.plot(z[i:i+N], np.polyval(coefficients, z[i:i+N]), linestyle='--', color=couleur, label=f'pente={pente:.2f}')
        # plt.errorbar(z[i:i+N], force[i:i+N], yerr=ecart_type, fmt='.', color=couleur, capsize=5)

        plt.xlabel('Coordonnée z mm', fontsize=16)
        plt.ylabel('Force (N)', fontsize=16)
        plt.grid(True)
        plt.legend()
    
print(f'pente n°{i} = {pentes} N/mm')
    # enregister en .txt
    # np.savetxt(dossier + savename, 'pentes')


plt.show()

pente n°48 = [['anche', 0, 'position y', -5.276, 'pente', 0.82], ['anche', 0, 'position y', -3.957, 'pente', 1.04], ['anche', 0, 'position y', -2.638, 'pente', 1.4], ['anche', 0, 'position y', -1.319, 'pente', 1.42], ['anche', 0, 'position y', 0.0, 'pente', 1.71], ['anche', 0, 'position y', 1.319, 'pente', 1.46], ['anche', 0, 'position y', 2.638, 'pente', 1.36], ['anche', 0, 'position y', 3.957, 'pente', 1.17], ['anche', 0, 'position y', 5.276, 'pente', 0.73], ['anche', 1, 'position y', -5.223, 'pente', 1.12], ['anche', 1, 'position y', -3.918, 'pente', 1.69], ['anche', 1, 'position y', -2.612, 'pente', 2.21], ['anche', 1, 'position y', -1.306, 'pente', 2.49], ['anche', 1, 'position y', 0.0, 'pente', 2.53], ['anche', 1, 'position y', 1.306, 'pente', 2.57], ['anche', 1, 'position y', 2.612, 'pente', 2.1], ['anche', 1, 'position y', 3.918, 'pente', 1.69], ['anche', 1, 'position y', 5.223, 'pente', 1.29], ['anche', 2, 'position y', -5.282, 'pente', 1.4], ['anche', 2, 'position y', -3.961,

In [12]:
plt.figure()
plt.plot(pentes[1], pentes[3])
plt.xlabel('Coordonnée z mm', fontsize=16)
plt.ylabel('Force (N)', fontsize=16)
plt.grid(True)
plt.legend()
plt.show()

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


## Traitement images

In [ ]:
# Charger l'image
# dossier = "Banc statique/2024.04.05/anche1/"
dossier = "2024.04.05/anche1/"
image = cv2.imread(dossier + 'image_0_0.jpg', cv2.IMREAD_GRAYSCALE)
# angle_rot = im_rotate(image)
# print(angle)

In [28]:
plt.close('all')

# Charger l'image
dossier = "Banc statique/2024.04.23/"
nom_anche = 'anche1/'
image = cv2.imread(dossier + nom_anche + 'image_0_0.jpg', cv2.IMREAD_GRAYSCALE)
angle_rot = im_rotate(image)
angle_rot = angle_rot[0]
# print(angle_rot)

# repère des bords de l'anche
cibles = ['Anche Gauche', 'Anche Droite']
hmax = 350  # hauteur de l'image

for R in range (9):
    for r in range (6):
        I = cv2.imread(dossier + nom_anche + f'image_{R}_{r}.jpg',  cv2.IMREAD_GRAYSCALE)
        # I_rotated = rotate(I, np.rad2deg(-angle_rot), mode='nearest')
        I = np.flipud(I) # Reverse the order of elements along axis 0
        I = I[150:hmax, :]  # couper l'image pour ne garder qu'une partie utile
        print(I)
        
        W = I.shape[1]  # Width
        H = I.shape[0]  # Height
        
        X = np.arange(1,W+1) 
        Y = np.arange(1,H+1)
        a = 30
        N = 20  # taille fenêtre glissante (en px)
        dist_px = 5

        # # Trouver les bords de l'anche, on gardera les indices pour toutes les anches mesurées de la série
        Ncibles = len(cibles)
        COLORS = plt.cm.cool(np.linspace(0, 1, Ncibles))

        Xcibles = np.full(Ncibles, np.nan)
        Ycibles = np.full(Ncibles, np.nan)

-0.00893558670552866
[[19 18 18 ... 14 14 14]
 [20 19 18 ... 14 14 15]
 [19 19 20 ... 15 15 15]
 ...
 [21 20 20 ... 18 18 18]
 [20 20 20 ... 18 18 18]
 [20 20 20 ... 18 18 18]]
[[18 18 18 ... 15 15 14]
 [18 18 18 ... 14 14 14]
 [18 19 20 ... 14 14 14]
 ...
 [19 19 19 ... 18 18 18]
 [20 19 19 ... 17 17 17]
 [21 20 20 ... 17 17 17]]
[[19 19 19 ... 14 14 14]
 [19 19 19 ... 14 14 14]
 [20 18 17 ... 15 15 15]
 ...
 [19 19 19 ... 18 18 17]
 [19 19 19 ... 18 18 17]
 [19 19 19 ... 19 18 18]]
[[19 19 18 ... 14 14 15]
 [19 18 18 ... 14 14 15]
 [18 19 18 ... 14 14 14]
 ...
 [21 19 18 ... 18 18 18]
 [21 20 18 ... 18 18 18]
 [19 19 19 ... 18 18 18]]
[[19 18 18 ... 15 14 14]
 [20 19 18 ... 14 14 14]
 [20 19 18 ... 14 14 14]
 ...
 [18 20 21 ... 17 18 18]
 [19 20 21 ... 18 18 18]
 [20 21 21 ... 18 18 18]]
[[20 20 20 ... 15 15 15]
 [19 19 19 ... 15 15 15]
 [19 19 19 ... 15 15 14]
 ...
 [20 21 21 ... 18 18 17]
 [20 20 20 ... 18 18 17]
 [21 21 21 ... 18 18 17]]
[[19 19 19 ... 14 14 15]
 [19 19 20 ... 14 

In [57]:
import cv2
import numpy as np

# Charger les images
image = cv2.imread(dossier + nom_anche + 'image_0_0.jpg', cv2.IMREAD_GRAYSCALE)
clone = image.copy()

# Définir une liste pour stocker les coordonnées des points
points = []
# Définir les coordonnées du milieu à l'extérieur de la fonction click_callback
milieu_x = None
milieu_y = None

# Définir une fonction de rappel pour la souris
def click_callback(event, x, y, flags, param):
    # Vérifier si un clic gauche a été effectué
    if event == cv2.EVENT_LBUTTONDOWN:
        # Ajouter les coordonnées du point à la liste
        points.append((x, y))
        # Dessiner un cercle sur l'image pour indiquer le point
        cv2.circle(clone, (x, y), 5, (0, 0, 255), -1)
        # Afficher l'image mise à jour
        cv2.imshow("Image", clone)

        # Si deux points ont été sélectionnés, calculer le milieu
        if len(points) == 2:
            milieu_x = (points[0][0] + points[1][0]) // 2
            milieu_y = (points[0][1] + points[1][1]) // 2
            # Dessiner un marqueur pour le milieu
            cv2.circle(clone, (milieu_x, milieu_y), 5, (0, 255, 0), -1)
            # Afficher l'image mise à jour
            cv2.imshow("Image", clone)

            # Afficher les coordonnées du milieu
            print("Coordonnées du milieu :")
            print(f"x = {milieu_x}, y = {milieu_y}")

            return milieu_x, milieu_y
            
            
# Créer une fenêtre pour afficher l'image
cv2.namedWindow("Image")
# Attacher la fonction de rappel à la fenêtre
cv2.setMouseCallback("Image", click_callback)

# Afficher l'image
cv2.imshow("Image", clone)
# Attendre que l'utilisateur appuie sur une touche
cv2.waitKey(0)
cv2.destroyAllWindows()

# Afficher les coordonnées des deux points
print("Coordonnées des points sélectionnés :")
for i, point in enumerate(points):
    print(f"Point {i+1}: {point}")
    
    

Coordonnées du milieu :
x = 1303, y = 564
Coordonnées des points sélectionnés :
Point 1: (1255, 564)
Point 2: (1352, 564)


TypeError: unsupported operand type(s) for -: 'NoneType' and 'int'

In [58]:
for R in range (9):
    for r in range (6):
        I = cv2.imread(dossier + nom_anche + f'image_{R}_{r}.jpg',  cv2.IMREAD_GRAYSCALE)
        
        # Calculer le déplacement du point d'intérêt par rapport à l'image d'origine
        deplacement_x = milieu_x - 100  # (par exemple, point d'intérêt original à la coordonnée x = 100)
        deplacement_y = milieu_y - 200  # (par exemple, point d'intérêt original à la coordonnée y = 200)

        # Nouvelles coordonnées du point d'intérêt sur l'image actuelle
        nouveau_point_x = milieu_x + deplacement_x
        nouveau_point_y = milieu_y + deplacement_y

        # Dessiner un marqueur pour le point d'intérêt sur l'image actuelle
        image_actuelle_coloree = cv2.cvtColor(I, cv2.COLOR_GRAY2BGR)
        cv2.circle(image_actuelle_coloree, (nouveau_point_x, nouveau_point_y), 5, (0, 255, 0), -1)

    # Afficher l'image avec le marqueur du point d'intérêt
    cv2.imshow(f"Image {i}", image_actuelle_coloree)

    # Attendre jusqu'à ce qu'une touche soit pressée
    cv2.waitKey(0)
    cv2.destroyAllWindows()

TypeError: unsupported operand type(s) for -: 'NoneType' and 'int'

In [1]:
# Charger l'image
dossier = "Banc statique/2024.04.05/anche1/"
image = cv2.imread(dossier + 'image_0_0.jpg')  # Remplacez 'votre_image.jpg' par le chemin de votre propre image

# Convertir l'image en niveaux de gris
image_grayscale = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Appliquer un flou gaussien pour réduire le bruit
image_blurred = cv2.GaussianBlur(image_grayscale, (5, 5), 0)

# Détection de contours avec l'algorithme de Canny
# contours = cv2.Canny(image_blurred, 50, 150)  # Les valeurs 50 et 150 sont des seuils min et max

# Appliquer un seuillage à l'image (si nécessaire)
_, thresholded_image = cv2.threshold(image_grayscale, 127, 255, cv2.THRESH_BINARY)

# Trouver les contours dans l'image seuillée
contours, _ = cv2.findContours(thresholded_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Dessiner les contours trouvés sur l'image d'origine
contoured_image = cv2.drawContours(image.copy(), contours, -1, (0, 255, 0), 2)

# Afficher l'image originale et les contours détectés
plt.figure(figsize=(12, 6))

# plt.subplot(1, 2, 1)
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
plt.title('Image originale')
plt.axis('off')

plt.subplot(1, 2, 2)
cv2.imshow('Contours', contoured_image)
# plt.imshow(contours, cmap='gray')
plt.title('Contours détectés')
plt.axis('off')

plt.show()


NameError: name 'cv2' is not defined

In [7]:
plt.close('all')
# Charger l'image
dossier = "Banc statique/2024.04.05/anche1/"
image = cv2.imread(dossier + 'image_0_0.jpg')  # Remplacez 'votre_image.jpg' par le chemin de votre propre image

# Convertir l'image en niveaux de gris
image_grayscale = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Appliquer le seuillage pour obtenir une image binaire (noir et blanc)
_, binary_image = cv2.threshold(image, 127, 255, cv2.THRESH_BINARY)

# Appliquer un flou gaussien pour réduire le bruit
image_blurred = cv2.GaussianBlur(image_grayscale, (5, 5), 0)

# Détection de contours avec l'algorithme de Canny
contours = cv2.Canny(image_blurred, 30, 90)  # Les valeurs 50 et 150 sont des seuils min et max

# Afficher l'image originale et les contours détectés
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
plt.title('Image originale')
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(binary_image, cmap='gray')
plt.title('Contours détectés')
plt.axis('off')

plt.show()
